# 📄 扫描件PDF → Markdown 转换工具

这个脚本用于将扫描件PDF文档自动转换为Markdown格式：
1. 提取PDF中的图像
2. 使用OCR识别文字（支持中英文）
3. 保留原始排版结构
4. 输出Markdown文件

## 工作流程
- **输入**: `input_pdfs/` 文件夹中的PDF文件
- **输出**: `output_markdown/` 文件夹中的Markdown文件

In [1]:
import os
import sys

import logging
import warnings

# ================= 强力静音设置 =================

# 1. 屏蔽 Paddle 底层 C++ 库的日志 (必须在 import paddle 前设置才最稳，但现在设也有用)
# 0=All, 1=No Info, 2=No Warning, 3=No Error
os.environ['GLOG_minloglevel'] = '2' 

# 2. 屏蔽 Python 标准 warnings (如 FutureWarning 等)
warnings.filterwarnings('ignore')

# 3. 【最关键一步】全局禁用 logging
# 这会强制屏蔽所有 logger 的 WARNING 及以下级别 (DEBUG, INFO, WARNING)
# 只有 ERROR 和 CRITICAL 才会显示
logging.disable(logging.WARNING) 

# ===============================================

# 将步骤 1 中获取的文件夹路径替换下方的 path_to_add
# 注意：Windows路径中的反斜杠需要使用双反斜杠 \\ 或在字符串前加 r
path_to_add = r"D:\CUDA\Development\bin\cudnn_ops_infer64_8.dll" 

if path_to_add not in os.environ['PATH']:
    os.environ['PATH'] = path_to_add + os.pathsep + os.environ['PATH']

# 验证路径添加后，再导入 paddle
import paddle
paddle.utils.run_check()


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


固定采取这样的版本组合：

python -m pip install --no-cache-dir paddlepaddle-gpu==2.6.2 -f https://pypi.tuna.tsinghua.edu.cn/simple -i https://pypi.tuna.tsinghua.edu.cn/simple
python -m pip install --no-cache-dir opencv-python==4.6.0.66 -i https://pypi.tuna.tsinghua.edu.cn/simple
python -m pip install --no-cache-dir imgaug==0.4.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
python -m pip install --no-cache-dir paddleocr==2.7.3 -i https://pypi.tuna.tsinghua.edu.cn/simple
python -m pip install --no-cache-dir numpy==1.26.4 -i https://pypi.tuna.tsinghua.edu.cn/simple
CUDA版本11.8
CUDNN需要从这里下载，解压后复制黏贴pip download nvidia-cudnn-cu11==8.9.7.29 -i https://pypi.tuna.tsinghua.edu.cn/simple
最后打上zlibwapi.dll

在PowerShell中将模型文件从默认下载位置复制到指定目录的命令如下：
'''PowerShell
PS :\WINDOWS\system32> Copy-Item -Path "C:\Users\赵子川\.paddleocr\whl\det\ch\ch_PP-OCRv4_det_infer\*" -Destination "D:\ocr_models\det\ch_PP-OCRv4_det_infer\" -Recurse -Force
PS C:\WINDOWS\system32> Copy-Item -Path "C:\Users\赵子川\.paddleocr\whl\rec\ch\ch_PP-OCRv4_rec_infer\*" -Destination "D:\ocr_models\rec\ch_PP-OCRv4_rec_infer\" -Recurse -Force
PS C:\WINDOWS\system32> Copy-Item -Path "C:\Users\赵子川\.paddleocr\whl\cls\ch_ppocr_mobile_v2.0_cls_infer\*" -Destination "D:\ocr_models\cls\ch_ppocr_mobile_v2.0_cls_infer\" -Recurse -Force
'''      

由于中文系统路径的问题，OCR在任何模型调用/初始化的时候都需要显式传入现在det等模型所处于的英文dir
'''python
det_model_dir=r'D:\\ocr_models\det\\ch_PP-OCRv4_det_infer',  # 显式传递det模型目录
rec_model_dir=r'D:\\ocr_models\\rec\\ch_PP-OCRv4_rec_infer',  # 显式传递rec模型目录
cls_model_dir=r'D:\\ocr_models\\cls\\ch_ppocr_mobile_v2.0_cls_infer',  # 显式传递cls模型目录
'''


In [2]:
# 导入必要的库
import os
import re
import sys
from pathlib import Path
from typing import List, Dict, Optional
import cv2
import numpy as np
from PIL import Image
import pymupdf  # PyMuPDF
import json
from datetime import datetime
from tqdm import tqdm

# OCR引擎
try:
    from paddleocr import PaddleOCR
    ocr = PaddleOCR(
        lang='ch',
        use_doc_orientation_classify=False,  
        use_doc_unwarping=False,
        det_model_dir=r'D:\\ocr_models\det\\ch_PP-OCRv4_det_infer',  # 显式传递det模型目录
        rec_model_dir=r'D:\\ocr_models\\rec\\ch_PP-OCRv4_rec_infer',  # 显式传递rec模型目录
        cls_model_dir=r'D:\\ocr_models\\cls\\ch_ppocr_mobile_v2.0_cls_infer',  # 显式传递cls模型目录             
        use_textline_orientation=False       
    )
    print("init ok")
    PADDLEOCR_AVAILABLE = True
except Exception as e:
    print(f"Error initializing PaddleOCR: {e}")
    PADDLEOCR_AVAILABLE = False

print("="*60)
print("🔧 初始化转换器")
print("="*60)
print(f"✓ PyMuPDF: {pymupdf.__version__}")
print(f"✓ OpenCV: {cv2.__version__}")
print(f"✓ PIL: {Image.__version__}")
print(f"✓ PaddleOCR: {'可用' if PADDLEOCR_AVAILABLE else '不可用'}")
print("="*60)

# 初始化OCR引擎
ocr_engine = None
ocr_type = None

if PADDLEOCR_AVAILABLE:
    print("\n初始化PaddleOCR（支持中英文）...")
    try:
        # 检测CUDA
        try:
            import torch
            cuda_available = torch.cuda.is_available()
            if cuda_available:
                print(f"  ✓ CUDA可用: {torch.cuda.get_device_name(0)}")
        except:
            cuda_available = False
        
        # 初始化PaddleOCR（新版本会自动检测CUDA）
        ocr_engine = PaddleOCR(
            use_textline_orientation=True, 
            det_model_dir=r'D:\\ocr_models\det\\ch_PP-OCRv4_det_infer',  # 显式传递det模型目录
            rec_model_dir=r'D:\\ocr_models\\rec\\ch_PP-OCRv4_rec_infer',  # 显式传递rec模型目录
            cls_model_dir=r'D:\\ocr_models\\cls\\ch_ppocr_mobile_v2.0_cls_infer',  # 显式传递cls模型目录   
            lang='ch'
        )
        ocr_type = 'paddleocr'
        print(f"✓ PaddleOCR初始化成功")
    except Exception as e:
        print(f"✗ PaddleOCR初始化失败: {e}")
        

if ocr_engine is None:
    print("\n❌ 错误: 未能初始化任何OCR引擎！")
    print("   请确保安装了 paddleocr")
    sys.exit(1)


init ok
🔧 初始化转换器
✓ PyMuPDF: 1.27.1
✓ OpenCV: 4.6.0
✓ PIL: 12.1.1
✓ PaddleOCR: 可用

初始化PaddleOCR（支持中英文）...
✓ PaddleOCR初始化成功


In [3]:
# 导入必要的库 (确保这些库已导入)
import os
import re
import sys
import cv2
import numpy as np
import pymupdf
from pathlib import Path
from typing import List, Dict
from datetime import datetime
from tqdm import tqdm

# 核心转换类 (更新版：支持硬盘中转与自动清理)
class PDFToMarkdownConverter:
    """将PDF扫描件转换为Markdown"""
    
    def __init__(self, 
                 input_dir: str = "input_pdfs",
                 output_dir: str = "output_markdown",
                 ocr_engine=None,
                 ocr_type: str = "paddleocr",
                 conf_threshold: float = 0.3,
                 save_images: bool = False):
        """初始化转换器"""
        self.input_dir = Path(input_dir)
        self.output_dir = Path(output_dir)
        self.ocr_engine = ocr_engine
        self.ocr_type = ocr_type
        self.conf_threshold = conf_threshold
        self.save_images = save_images # 如果为True，则不删除图片；如果为False，则处理完即删
        
        self.output_dir.mkdir(parents=True, exist_ok=True)
        # 创建一个专门的临时图片目录，避免污染根目录
        self.temp_img_dir = self.output_dir / "temp_images"
        self.temp_img_dir.mkdir(parents=True, exist_ok=True)
        
        self.stats = {
            'total_pdfs': 0,
            'successful': 0,
            'failed': 0,
            'total_pages': 0,
            'total_blocks': 0
        }
    
    def extract_pdf_pages(self, pdf_path: str) -> List[Dict]:
        """从PDF提取页面和图像"""
        pages = []
        try:
            pdf_doc = pymupdf.open(pdf_path)
            
            for page_num in range(len(pdf_doc)):
                page = pdf_doc[page_num]
                page_info = {
                    'page_num': page_num + 1,
                    'width': page.rect.width,
                    'height': page.rect.height,
                    'image': None,
                    'text': None
                }
                
                # Matrix(3, 3) 保证了高分辨率，这是识别成功的物理基础
                pix = page.get_pixmap(matrix=pymupdf.Matrix(3, 3))
                img_data = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.height, pix.width, pix.n)
                
                # 颜色空间转换
                if pix.n == 4:
                    img_data = cv2.cvtColor(img_data, cv2.COLOR_RGBA2BGR)
                elif pix.n == 3:
                    img_data = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
                
                page_info['image'] = img_data
                pages.append(page_info)
            
            pdf_doc.close()
            self.stats['total_pages'] += len(pages)
            
        except Exception as e:
            print(f"❌ 提取PDF失败: {e}")
        
        return pages
    
    def perform_ocr(self, image_input) -> str:
        """
        对图像进行OCR识别
        image_input: 可以是 numpy 数组，也可以是图片路径(str)
        本次修改主要利用 str 路径来传参
        """
        if self.ocr_engine is None:
            return ""
        
        try:
            if self.ocr_type == 'paddleocr':
                # PaddleOCR 自动判断：如果是字符串就读文件，如果是数组就读内存
                results = self.ocr_engine.ocr(image_input)
                text_lines = []
                
                # 增强的空结果检查
                if results and len(results) > 0 and results[0] is not None:
                    for line in results[0]:
                        text = line[1][0]
                        score = line[1][1]
                        if score >= self.conf_threshold:
                            text_lines.append(text)
                
                return "\n".join(text_lines)
            
            # (RapidOCR 逻辑省略，保持原样或按需修改)
            return ""
        
        except Exception as e:
            print(f"⚠ OCR识别出错: {e}")
            return ""
    
    def format_markdown(self, pages: List[Dict], pdf_name: str) -> str:
        """格式化为Markdown（修复文件夹被意外删除的问题）"""
        
        # 清洗文件名
        safe_pdf_name = re.sub(r'[\\/:*?"<>|()\s]', '_', pdf_name)
        
        md_lines = []
        md_lines.append(f"# {pdf_name}")
        md_lines.append(f"\n*转换时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n")
        md_lines.append("---")
        
        # 【重要修复】确保临时目录在每一页处理前都存在 (防止被误删)
        if not self.temp_img_dir.exists():
            self.temp_img_dir.mkdir(parents=True, exist_ok=True)
        
        print(f"\n[DEBUG] 准备处理 {len(pages)} 页...")

        for page_info in tqdm(pages, desc="OCR识别进度", leave=False):
            page_num = page_info['page_num']
            image = page_info['image']
            
            md_lines.append(f"## 第 {page_num} 页\n")
            
            if image is None:
                md_lines.append("*[图像数据丢失]*")
                continue
            
            img_filename = f"{safe_pdf_name}_p{page_num}.jpg"
            img_path = self.temp_img_dir / img_filename
            img_path_str = str(img_path)
            
            try:
                is_success, buffer = cv2.imencode(".jpg", image)
                if is_success:
                    buffer.tofile(img_path_str)
                else:
                    print(f"❌ 图片编码失败: {img_filename}")
                    continue
            except Exception as e:
                # 这里会打印你遇到的 Errno 2
                print(f"❌ 写入临时文件异常: {e}")
                continue

            # OCR 识别
            text = self.perform_ocr(img_path_str)
            
            # 自动清理单张图片
            if not self.save_images:
                try:
                    if os.path.exists(img_path_str):
                        os.remove(img_path_str)
                except Exception as e:
                    pass 
            
            if text:
                md_lines.append(text)
            else:
                md_lines.append("*[无法识别文本]*")
            
            md_lines.append("\n---\n")
            self.stats['total_blocks'] += 1
        
        return "\n".join(md_lines)
    
    def convert_pdf(self, pdf_file: str) -> bool:
        """转换单个PDF文件"""
        try:
            pdf_path = self.input_dir / pdf_file
            
            if not pdf_path.exists():
                print(f"❌ 文件不存在: {pdf_file}")
                return False
            
            print(f"\n📄 处理: {pdf_file}")
            pages = self.extract_pdf_pages(str(pdf_path))
            if not pages:
                print(f"❌ 无法提取页面")
                return False
            
            pdf_name = Path(pdf_file).stem
            markdown_content = self.format_markdown(pages, pdf_name)
            
            output_file = self.output_dir / f"{pdf_name}.md"
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(markdown_content)
            
            print(f"✓ 已保存: {output_file}")
            self.stats['successful'] += 1
            return True
            
        except Exception as e:
            print(f"❌ 转换失败: {e}")
            self.stats['failed'] += 1
            return False
    
    def convert_all(self) -> Dict:
        """转换输入文件夹中的所有PDF"""
        if not self.input_dir.exists():
            print(f"❌ 输入文件夹不存在: {self.input_dir}")
            return self.stats
        
        pdf_files = list(self.input_dir.glob("*.pdf"))
        self.stats['total_pdfs'] = len(pdf_files)
        
        if not pdf_files:
            print(f"⚠ 输入文件夹中没有PDF文件")
            return self.stats
        
        print(f"\n📊 开始转换 {len(pdf_files)} 个PDF文件...")
        print("="*60)
        
        for pdf_file in tqdm(pdf_files, desc="转换进度"):
            self.convert_pdf(pdf_file.name)
        
        return self.stats
    
    def print_stats(self):
        print("\n" + "="*60)
        print("📈 转换统计")
        print("="*60)
        print(f"总PDF数: {self.stats['total_pdfs']}")
        print(f"成功: {self.stats['successful']}")
        print(f"失败: {self.stats['failed']}")
        print(f"总页数: {self.stats['total_pages']}")
        print(f"总文本块: {self.stats['total_blocks']}")
        print("="*60 + "\n")

print("✓ 转换器类已更新 (硬盘中转 + 自动清理版)")

✓ 转换器类已更新 (硬盘中转 + 自动清理版)


In [4]:
# 创建转换器实例并运行
print("\n🚀 创建转换器...")

# 确定工作目录
work_dir = Path.cwd()
input_dir = work_dir / "input_pdf"
output_dir = work_dir / "output_markdown"

print(f"工作目录: {work_dir}")
print(f"输入目录: {input_dir}")
print(f"输出目录: {output_dir}")

# 创建转换器
converter = PDFToMarkdownConverter(
    input_dir=str(input_dir),
    output_dir=str(output_dir),
    ocr_engine=ocr_engine,
    ocr_type=ocr_type,
    conf_threshold=0.3,
    save_images=False  
)

print("✓ 转换器已就绪\n")

# 显示使用说明
print("="*60)
print("📋 使用说明")
print("="*60)
print(f"""
1. 将PDF文件放入: {input_dir}/
2. 运行下一个单元格中的转换命令:
   
   stats = converter.convert_all()
   converter.print_stats()

3. 转换完成后，Markdown文件将保存在:
   {output_dir}/
""")
print("="*60)



🚀 创建转换器...
工作目录: e:\量子波动速读预处理
输入目录: e:\量子波动速读预处理\input_pdf
输出目录: e:\量子波动速读预处理\output_markdown
✓ 转换器已就绪

📋 使用说明

1. 将PDF文件放入: e:\量子波动速读预处理\input_pdf/
2. 运行下一个单元格中的转换命令:
   
   stats = converter.convert_all()
   converter.print_stats()

3. 转换完成后，Markdown文件将保存在:
   e:\量子波动速读预处理\output_markdown/



## 🚀 执行转换

在下面的单元格中运行转换命令

In [5]:

# 执行转换
stats = converter.convert_all()
converter.print_stats()



📊 开始转换 4 个PDF文件...


转换进度:   0%|          | 0/4 [00:00<?, ?it/s]


📄 处理: 人地之间：中国增长模式下的城乡土地改革 (陶然) (Z-Library).pdf

[DEBUG] 准备处理 396 页...


转换进度:  25%|██▌       | 1/4 [03:24<10:13, 204.41s/it]

✓ 已保存: e:\量子波动速读预处理\output_markdown\人地之间：中国增长模式下的城乡土地改革 (陶然) (Z-Library).md

📄 处理: 当代中国政府与政治 (景跃进 陈明明 肖滨) (Z-Library).pdf

[DEBUG] 准备处理 293 页...


...

✓ 已保存: e:\量子波动速读预处理\output_markdown\当代中国政府与政治 (景跃进 陈明明 肖滨) (Z-Library).md

📄 处理: 当代中国的国家与社会关系 (周雪光 主编) (Z-Library).pdf

[DEBUG] 准备处理 305 页...


转换进度:  75%|███████▌  | 3/4 [09:57<03:11, 191.39s/it]

✓ 已保存: e:\量子波动速读预处理\output_markdown\当代中国的国家与社会关系 (周雪光 主编) (Z-Library).md

📄 处理: 新中国财政金融制度变迁事件解读 (陈雨露 郭庆旺) (Z-Library).pdf

[DEBUG] 准备处理 516 页...


转换进度: 100%|██████████| 4/4 [16:37<00:00, 249.40s/it]

✓ 已保存: e:\量子波动速读预处理\output_markdown\新中国财政金融制度变迁事件解读 (陈雨露 郭庆旺) (Z-Library).md

📈 转换统计
总PDF数: 4
成功: 4
失败: 0
总页数: 1510
总文本块: 1510

